In [34]:
import pandas as pd
import numpy as np
import datetime

In [6]:
#文件列表
pre_path='data/pred/'
filelist=['B_RandomForestRegressor_20180925094702.csv',
        'B_GradientBoostingRegressor_20180925111540.csv',
         'B_XGBRegressor_20180925130757_ceil.csv']

## 结果合并规则
- 有两个值一样的,则以一样的值为准
- 三个值都不一样,则取平均值

In [30]:
df1 = pd.read_csv(pre_path+"B_RandomForestRegressor_20180925094702.csv")
df2 = pd.read_csv(pre_path+"B_GradientBoostingRegressor_20180925111540.csv")
df3 = pd.read_csv(pre_path+"B_XGBRegressor_20180925125403.csv")

df1.rename(columns={'car_number':'rf_car_number'}, inplace=True)
df2.rename(columns={'car_number':'gb_car_number'}, inplace=True)
df3.rename(columns={'car_number':'xgb_car_number'}, inplace=True)
df3['xgb_car_number']=df3['xgb_car_number'].astype(np.int64)

df = pd.concat([df1,df2['gb_car_number'],df3['xgb_car_number']], axis=1)


In [32]:
df.head()

,grid_id,day,hour,rf_car_number,gb_car_number,xgb_car_number
0,51,20170313,9,8,8,7
1,52,20170313,9,0,1,0
2,53,20170313,9,3,4,3
3,54,20170313,9,6,6,5
4,55,20170313,9,5,6,6


In [38]:
def merge_result(row):
    if row['rf_car_number'] == row['gb_car_number']:
        return row['gb_car_number']
    if row['rf_car_number'] == row['xgb_car_number']:
        return row['xgb_car_number']
    if row['gb_car_number'] == row['xgb_car_number']:
        return row['xgb_car_number']
    return np.ceil((row['rf_car_number']+row['gb_car_number']+row['xgb_car_number'])/3)

df['car_number']=df.apply(merge_result, axis=1)


In [41]:
now_time = datetime.datetime.now()

df[['grid_id','day','hour','car_number']].to_csv(pre_path+"B_merge_{}.csv".format(now_time.strftime('%Y%m%d%H%M%S')), index=False)